In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
# tf.config.set_logical_device_configuration(
# gpus[0],
# [tf.config.LogicalDeviceConfiguration(memory_limit=9216  )])

2022-03-23 15:23:45.592681: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 15:23:45.609831: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 15:23:45.609904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5,
                                                             from_pt=True)

2022-03-23 15:23:54.622757: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-23 15:23:54.624121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 15:23:54.624270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 15:23:54.624353: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [3]:
from datasets import load_dataset, DatasetDict

raw_dataset = DatasetDict.load_from_disk("./data")

In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', '__index_level_0__'],
        num_rows: 23944
    })
    test: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', '__index_level_0__'],
        num_rows: 5987
    })
})

In [4]:
def tokenize(example):
    tokenized_example = tokenizer(example["Opinion"], truncation=True)
    tokenized_example["label"] = example["Polarity"]
    return tokenized_example


tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=raw_dataset["train"].column_names)

Loading cached processed dataset at data/train/cache-9de50fbf96ed39da.arrow


  0%|          | 0/6 [00:00<?, ?ba/s]

In [6]:
tokenized_dataset["train"][0]

{'input_ids': [0,
  50229,
  405,
  727,
  28091,
  909,
  3269,
  129,
  334,
  344,
  9801,
  9286,
  16758,
  342,
  2608,
  16398,
  6767,
  313,
  1216,
  389,
  1450,
  369,
  834,
  68,
  1893,
  719,
  21560,
  3424,
  367,
  466,
  1509,
  8335,
  1045,
  3108,
  68,
  275,
  68,
  17500,
  334,
  332,
  7414,
  3370,
  3223,
  28091,
  394,
  2070,
  68,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'label': 4}

In [5]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [6]:
batch_size = 8

tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=batch_size,
)

tf_eval_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=batch_size,
)

In [9]:
model.config.num_labels

5

In [7]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060, compute capability 8.6


2022-03-23 15:24:13.574719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [8]:
from transformers import create_optimizer

num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [9]:
from transformers.keras_callbacks import PushToHubCallback
callback = PushToHubCallback(output_dir="classificationEsp2", tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/mperez/nlp/subject_competition/RoBERTaESP/classificationEsp2 is already a clone of https://huggingface.co/Zarkit/classificationEsp2. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
import numpy as np
from datasets import load_metric
 
load_accuracy = load_metric("accuracy")
load_f1 = load_metric("f1")
for batch in tf_eval_dataset:
    logits = model.predict(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    load_accuracy.add_batch(predictions=predictions, references=labels)
    load_f1.add_batch(predictions=predictions, references=labels)


accuracy = load_accuracy.compute()["accuracy"]
f1 = load_f1.compute(average="None")["f1"]

In [ ]:
print(f'Accuracy: {accuracy:.2f}')
print(f'F1: {f1:.2f}')


In [ ]:
model.push_to_hub("classificationEsp1")
tokenizer.push_to_hub("classificationEsp1")

In [5]:
# Tests ------

check_point = "Zarkit/classificationEsp1"

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(check_point)
tokenizer = AutoTokenizer.from_pretrained(check_point)


from datasets import load_dataset, DatasetDict

raw_dataset = DatasetDict.load_from_disk("./data")

def tokenize(example):
    tokenized_example = tokenizer(example["Opinion"], truncation=True)
    tokenized_example["label"] = example["Polarity"]
    return tokenized_example


tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=raw_dataset["train"].column_names)


from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")


tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

import numpy as np
from datasets import load_metric
 
load_accuracy = load_metric("accuracy")
load_f1 = load_metric("f1")
for batch in tf_eval_dataset:
    logits = model.predict(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    load_accuracy.add_batch(predictions=predictions, references=labels)
    load_f1.add_batch(predictions=predictions, references=labels)


accuracy = load_accuracy.compute()["accuracy"]
f1 = load_f1.compute(average=None)["f1"]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at Zarkit/classificationEsp1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [7]:
print(f'Accuracy: {accuracy:.2f}')
print(f'F1: {f1}')


Accuracy: 0.77
F1: [0.50381679 0.34       0.48       0.47850854 0.88372718]


In [8]:
raw_dataset["test"]["Opinion"][0:3]

['El mejor lugar para conocer los dulces típicos y artesanías de la ciudad a un precio accesible. La comida a un costado es buena más no es lo mejor que la hermosa ciudad de Guanajuato te puede ofrecer.',
 'Esto debe de haber sido nuestro 10Â° o así que visita para cenar en los últimos cinco años y el servicio y la calidad de la comida ha estado alto en todo. Pregunta por Javier o Héctor a ti y no te equivocarás.\nColinbn (alias inglés Eddy), Toronto.',
 'Restaurante un paño blanco de mesa con excelente servicio y precio muy razonable. Mi y yo compartimos una fritto misto de mariscos y cada una tenía una pasta de mariscos. Las raciones de marisco con cada plato era grande y el clam piezas eran enormes en la pasta estafador vongole que se sirve.']